In [1]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from time import perf_counter
import sys
sys.path.append('/_Scripts')
from _Scripts import timehorizonestimator
from _Scripts import functionapproximator
# convert to dataframe
df = [pd.read_csv("datasets/phoenixdata.csv"), pd.read_csv("datasets/flagstaffdata.csv")]

# input variables
dataset_num = 0
p = [[],[],[],[],[]]
xp = [[],[],[],[]] 
agg_x = [[],[],[],[]] 
agg_mu = [[],[],[],] 
g_cuttoff = 25

# statistics visualization
g_ps = [[],[]]
g_slopes = []
tempdiff_arr = []
g_mtrs = []
avg_mu = []
avg_lambda = []
mu_diff = [[],[]]
lambda_diff = []
day_arr = list(range(0, 364))
# end


def first_order_model(data, timehorizon):
    day = 0
    temperatureDayArr = []
    GHIDayArr = []
    for i in np.arange(int(len(data['Minute']))):
        if (data['Minute'][i] % 1440 == 0):
            if (day > 0):
                t0, t3 = timehorizonestimator.findt0t2t3(
                    temperatureDayArr, g_cuttoff)
                t2 = int(1/3*t0 + 2/3 * t3)
                g_ps[dataset_num].append((t3-t0)/10)

                sensitivity_analysis(max(temperatureDayArr),
                                     g_cuttoff)

                mut0t2 = 0
                sum_lambda = 0
                netflow = 0
                netflowt2t3 = 0
                sum_netflow = 0
                sum_mu = 0
                sum_lambda = 0
                slope = 1  # guessed
                eqn_queue = []
                mu_arr = []

                # t0 t2
                for k in np.arange(int(t0/timehorizon), int(t2/timehorizon)+1):
                    mut0t2 = ((GHIDayArr[int(t2/timehorizon)]-GHIDayArr[int(t0/timehorizon)])/(
                        (t2-t0)/timehorizon))*(k-t0/timehorizon)+GHIDayArr[int(t0/timehorizon)]
                    mu_arr.append(mut0t2)
                    sum_mu += mut0t2
                    netflow = GHIDayArr[int(k)] - mut0t2
                    sum_netflow += netflow

                queue = sum_netflow

                # total lambda for t2 t3
                for k in np.arange(int(t2/timehorizon), int(t3/timehorizon)+1):
                    sum_lambda += GHIDayArr[int(k)]

                # approximate slope
                functionapproximator.func_approx(slope, sum_netflow, GHIDayArr[int(int(t2/timehorizon))], GHIDayArr, sum_lambda, t2, t3, timehorizon, .2, g_slopes)

                # total lambda for t2 t3
                for k in np.arange(int(t0/timehorizon), int(t2/timehorizon)+1):
                    sum_lambda += GHIDayArr[int(k)]

                slope = g_slopes[len(g_slopes) - 1]

                # t2 - t3
                for k in np.arange(0, int(t3/timehorizon)-int(t2/timehorizon)):
                    y = slope*k + GHIDayArr[int(int(t2/timehorizon))]
                    sum_mu += y
                    netflowt2t3 = GHIDayArr[int(k) + int(t2/timehorizon)] - y
                    queue += netflowt2t3

                avg_mu.append((1 / (t3-t0)) * sum_mu)
                avg_lambda.append((1 / (t3-t0)) * sum_lambda)
                # eqn 1.1 (given)
                for k in np.arange(int(t0/timehorizon), int(t3/timehorizon)+1):
                    y = 1/3*((k - int(t0/timehorizon))**2) * \
                        (int(t3/timehorizon) - k)
                    eqn_queue.append(y)
                    mu_arr.append(y)
                mu_diff[dataset_num].append(
                    mu_arr[int(t2/timehorizon)-int(t0/timehorizon)] - mu_arr[0])
                lambda_diff.append(
                    GHIDayArr[int(t2/timehorizon)] - GHIDayArr[int(t0/timehorizon)])

                # print(eqn_queue)
                #print("queue t2 t3: " + str(queue))
                #print("queue t0 t2: " + str(sum_netflow))

            temperatureDayArr.clear()
            GHIDayArr.clear()
            day += 1
        temperatureDayArr.append(data['Average of Temperature'][i])
        GHIDayArr.append(data['Average of GHI'][i])


def sensitivity_analysis(maxtemp, cutoff):
    # mtr = magnitude of temperature reduction
    tempdiff = maxtemp - cutoff
    mtr = tempdiff/cutoff
    tempdiff_arr.append(tempdiff)
    g_mtrs.append(mtr)

def trim_values(bottom, top, val1, val2):
    #
    # print(bottom)
    temp1 = []
    temp2 = []
    for i in range(len(val1)):
        if (i >= bottom and i <= top):       
            temp1.append(val1[i])
            temp2.append(val2[i])
    return temp1, temp2
            
def plot_p():
    # Plot inflow rate
    fig = plt.figure()
    plt.plot(g_ps[0], 'r-', linewidth=3, label='Inflow rate')
    plt.ylabel('Hot Period (p)', fontsize=12)
    plt.xlabel('Day', fontsize=12)
    fig.savefig('Figures/p_plot.png', dpi=300, bbox_inches='tight')
    plt.show()

# inputs: x and input_val
# input value


def kdaysmoothing(x, input_val):
    agg_x = []
    y = []
    for i in np.arange(len(x)):

        total = 0
        average = 0
        count = 0
        for k in np.arange(len(x)):
            if (x[i] == x[k]):
                total += input_val[k]
                count += 1
        # since count is greater than 0, p value must exist
        if (count > 0):
            average = total / count
            dupe = False
            for j in range(len(y)):
                if (average == y[j]):
                    dupe = True
            if (dupe == False):
                y.append(average)
                agg_x.append(x[i])
    return agg_x, y


def plot_mtr():
    # Plot inflow rate
    ps, agg_mtr_vals = kdaysmoothing(g_ps[0], g_mtrs)

    cubic_fit = np.polyfit(ps, agg_mtr_vals, 2)
    p = np.poly1d(cubic_fit)
    xp = np.linspace(0, 144, 100)

    fig = plt.figure()

    plt.plot(g_ps[0], g_mtrs, '.', color='black', markersize=3)
    plt.scatter(ps, agg_mtr_vals, s=30, facecolors='none', edgecolors='r')

    plt.plot(xp, p(xp), '-', label='lienar')
    plt.legend(['Data', 'Mean Value Of Same P',
               'Estimated Curve'], loc='upper left')

    plt.xlabel('Hot Period (p)', fontsize=12)
    plt.ylabel('Magnitude Of Temperature Reduction', fontsize=12)
    plt.grid()

    plt.show()
    fig.savefig('Figures/mtr_vs_p.svg')

def findbestfit(x,i):
    aggx, aggmu = kdaysmoothing(x, mu_diff[i])
    
    #print(aggx)
    
    cubic_fit = np.polyfit(aggx, aggmu, 2)
    p = np.poly1d(cubic_fit)
    xp = np.linspace(0, max(x), 500)
    return p,xp,aggx,aggmu

def plot_mu(x, x_var_name, p, xp, agg_x,agg_mu):
    


    fig = plt.figure()
    for i in range(len(df)):
        #plt.plot(x[i], mu_diff[i], '.', color='black', markersize=3)
        #plt.scatter(agg_x[i], agg_mu[i], s=30, facecolors='none', edgecolors='r')

        plt.plot(xp[i], p[i](xp[i]), '-')
        plt.legend(['Data', 'Mean Value Of Same ' + x_var_name,
               'Estimated Curve'], loc='upper left')

        plt.title('Difference Mu ' + 'vs ' + x_var_name)
        plt.xlabel(x_var_name, fontsize=12)
        plt.ylabel('Difference Mu', fontsize=12)
    
    plt.grid()

    plt.show()
    fig.savefig('Figures/avg_mu' + str(id(x)) + '.png')
    # approximates function from area bound and lambda(t)



if __name__ == "__main__":
    x_value = [day_arr,day_arr]
    
    t1_start = perf_counter()
    for i in range(len(df)):
        first_order_model(df[i],10)
        dataset_num += 1
        p[i], xp[i], agg_x[i],agg_mu[i] = findbestfit(x_value[i],i)
        
    
    t1_stop = perf_counter()
    # plot_p()
    # plot_mtr()
    #day_arr, mu_diff[dataset_num] = trim_values(0,364, day_arr, mu_diff[dataset_num])
    #print(p)
    plot_mu(x_value, "Day",p,xp,agg_x,agg_mu)
    print("Elapsed time:", t1_stop - t1_start)


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/data3.csv'